In [1]:
!pip install ultralytics > /dev/null

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
from ultralytics.nn.modules import Conv, C2f, SPPF, Detect
from ultralytics.utils import LOGGER
from ultralytics.models.yolo.detect import DetectionTrainer
from ultralytics.utils.torch_utils import select_device, smart_inference_mode
import numpy as np
from typing import Dict, List, Tuple, Optional, Any, Union
import math
import os
import sys
from pathlib import Path
import yaml
import shutil
from sklearn.model_selection import train_test_split
from torch.optim import AdamW, SGD
import warnings
import logging
import gc
import time
from scipy import ndimage
import cv2
from PIL import Image
warnings.filterwarnings('ignore')

# Глобальная переменная для режима отладки
# Если True - обучение на 800 изображениях, 1 эпоха, валидация на всех данных
# Если False - обучение на всех данных
IS_DEBUG = True

# Пути к датасетам Kaggle
TRAIN_DATASET_1 = '/kaggle/input/01trains1datasethumanrescu1'
TRAIN_DATASET_2 = '/kaggle/input/02secondpartdatasethumanrescue'
VAL_DATASET_PUBLIC = '/kaggle/input/03validationdatasethumanrescue/public'
VAL_DATASET_PRIVATE = '/kaggle/input/03validationdatasethumanrescue/private'

log_dir = "/kaggle/working/"

logger = logging.getLogger('ml')
logger.setLevel(logging.DEBUG)

# Prevent adding handlers multiple times
if logger.hasHandlers():
    logger.handlers.clear()

# File handlers for different log levels
from logging.handlers import RotatingFileHandler

# Error log handler
error_handler = RotatingFileHandler(
    os.path.join(log_dir, 'main_error.log'),
    maxBytes=1*1024*1024,  # 1MB
    backupCount=5,
    encoding='utf-8'
)
error_handler.setLevel(logging.ERROR)

# Warning log handler
warning_handler = RotatingFileHandler(
    os.path.join(log_dir, 'main_warning.log'),
    maxBytes=1*1024*1024,  # 1MB
    backupCount=5,
    encoding='utf-8'
)
warning_handler.setLevel(logging.WARNING)

# Debug log handler
debug_handler = RotatingFileHandler(
    os.path.join(log_dir, 'main_debug.log'),
    maxBytes=1*1024*1024,  # 1MB
    backupCount=5,
    encoding='utf-8'
)
debug_handler.setLevel(logging.DEBUG)

# Console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Formatter
formatter = logging.Formatter(
    '%(asctime)s - MAIN - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s'
)
error_handler.setFormatter(formatter)
warning_handler.setFormatter(formatter)
debug_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers
logger.addHandler(error_handler)
logger.addHandler(warning_handler)
logger.addHandler(debug_handler)
logger.addHandler(console_handler)

logger.info("Logging setup completed")
print("[DEBUG] Logging setup completed successfully.")

def create_combined_dataset_yaml(output_path: str = 'combined_data.yaml') -> str:
    """Создание YAML конфигурации для объединенных датасетов"""
    
    # В режиме отладки создаем временную структуру
    if IS_DEBUG:
        logger.info("🐛 Создание YAML для debug режима")
        
        # Создание debug структуры
        train_datasets = [TRAIN_DATASET_1, TRAIN_DATASET_2]
        debug_train_dir = create_debug_dataset_structure(train_datasets, 800)
        
        if debug_train_dir:
            yaml_config = {
                'path': '/kaggle/working/debug_dataset',
                'train': os.path.join(debug_train_dir, 'images'),
                'val': os.path.join(VAL_DATASET_PUBLIC, 'images') if os.path.exists(VAL_DATASET_PUBLIC) else '',
                'test': os.path.join(VAL_DATASET_PRIVATE, 'images') if os.path.exists(VAL_DATASET_PRIVATE) else '',
                'nc': 1,
                'names': ['person']
            }
            
            # Сохранение YAML файла
            try:
                with open(output_path, 'w', encoding='utf-8') as f:
                    yaml.dump(yaml_config, f, default_flow_style=False, allow_unicode=True)
                logger.info(f"✅ Debug YAML конфигурация сохранена: {output_path}")
                logger.info(f"   Обучение: {yaml_config['train']}")
                logger.info(f"   Валидация: {yaml_config['val']}")
                return output_path
            except Exception as e:
                logger.error(f"❌ Ошибка сохранения debug YAML: {e}")
                return ''
    
    # Обычный режим - проверка существования датасетов
    datasets_info = {
        'train_1': {'path': TRAIN_DATASET_1, 'exists': False},
        'train_2': {'path': TRAIN_DATASET_2, 'exists': False},
        'val_public': {'path': VAL_DATASET_PUBLIC, 'exists': False},
        'val_private': {'path': VAL_DATASET_PRIVATE, 'exists': False}
    }
    
    for name, info in datasets_info.items():
        if os.path.exists(info['path']):
            info['exists'] = True
            images_path = os.path.join(info['path'], 'images')
            labels_path = os.path.join(info['path'], 'labels')
            if os.path.exists(images_path) and os.path.exists(labels_path):
                # Проверяем структуру датасета
                subdirs = [d for d in os.listdir(images_path) if os.path.isdir(os.path.join(images_path, d))]
                
                if subdirs:  # Иерархическая структура
                    image_label_pairs = collect_hierarchical_images(info['path'])
                    image_count = len(image_label_pairs)
                    label_count = len([pair for pair in image_label_pairs if os.path.exists(pair[1])])
                    logger.info(f"📊 {name} (иерархическая): {image_count} изображений, {label_count} меток в {len(subdirs)} подпапках")
                else:  # Плоская структура
                    image_count = len([f for f in os.listdir(images_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                    label_count = len([f for f in os.listdir(labels_path) if f.lower().endswith('.txt')])
                    logger.info(f"📊 {name} (плоская): {image_count} изображений, {label_count} меток")
            else:
                logger.warning(f"⚠️ {name}: отсутствуют папки images или labels")
        else:
            logger.warning(f"⚠️ {name}: датасет не найден по пути {info['path']}")
    
    # Создание конфигурации YAML
    yaml_config = {
        'path': os.path.dirname(os.path.abspath(output_path)),
        'train': [],
        'val': os.path.join(VAL_DATASET_PUBLIC, 'images') if datasets_info['val_public']['exists'] else '',
        'test': os.path.join(VAL_DATASET_PRIVATE, 'images') if datasets_info['val_private']['exists'] else '',
        'nc': 1,
        'names': ['person']
    }
    
    # Добавление обучающих датасетов
    if datasets_info['train_1']['exists']:
        yaml_config['train'].append(os.path.join(TRAIN_DATASET_1, 'images'))
    if datasets_info['train_2']['exists']:
        yaml_config['train'].append(os.path.join(TRAIN_DATASET_2, 'images'))
    
    # Если только один датасет, убираем список
    if len(yaml_config['train']) == 1:
        yaml_config['train'] = yaml_config['train'][0]
    elif len(yaml_config['train']) == 0:
        logger.error("❌ Не найдено ни одного обучающего датасета!")
        yaml_config['train'] = ''
    
    # Сохранение YAML файла
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            yaml.dump(yaml_config, f, default_flow_style=False, allow_unicode=True)
        logger.info(f"✅ YAML конфигурация сохранена: {output_path}")
        return output_path
    except Exception as e:
        logger.error(f"❌ Ошибка сохранения YAML: {e}")
        return ''

def collect_hierarchical_images(dataset_path: str) -> List[Tuple[str, str]]:
    """
    Сбор всех пар изображение-метка из иерархической структуры датасета
    
    Args:
        dataset_path: Путь к датасету с подпапками
        
    Returns:
        List[Tuple[str, str]]: Список пар (путь_к_изображению, путь_к_метке)
    """
    image_label_pairs = []
    
    images_base = os.path.join(dataset_path, 'images')
    labels_base = os.path.join(dataset_path, 'labels')
    
    if not (os.path.exists(images_base) and os.path.exists(labels_base)):
        logger.warning(f"⚠️ Не найдены папки images или labels в {dataset_path}")
        return image_label_pairs
    
    # Получение списка подпапок в images
    try:
        image_subdirs = [d for d in os.listdir(images_base) 
                        if os.path.isdir(os.path.join(images_base, d))]
        image_subdirs = sorted(image_subdirs)
        
        logger.info(f"📁 Найдено {len(image_subdirs)} подпапок в {images_base}: {image_subdirs}")
        
        for subdir in image_subdirs:
            images_subdir = os.path.join(images_base, subdir)
            labels_subdir = os.path.join(labels_base, subdir)
            
            if not os.path.exists(labels_subdir):
                logger.warning(f"⚠️ Не найдена соответствующая папка меток: {labels_subdir}")
                continue
            
            # Сбор изображений из подпапки
            try:
                image_files = [f for f in os.listdir(images_subdir) 
                             if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                
                for img_file in image_files:
                    img_path = os.path.join(images_subdir, img_file)
                    
                    # Поиск соответствующей метки
                    label_name = os.path.splitext(img_file)[0] + '.txt'
                    label_path = os.path.join(labels_subdir, label_name)
                    
                    if os.path.exists(label_path):
                        image_label_pairs.append((img_path, label_path))
                    else:
                        logger.warning(f"⚠️ Не найдена метка для {img_file}: {label_path}")
                        
            except Exception as e:
                logger.error(f"❌ Ошибка обработки подпапки {subdir}: {e}")
                continue
        
        logger.info(f"✅ Собрано {len(image_label_pairs)} пар изображение-метка из {dataset_path}")
        
    except Exception as e:
        logger.error(f"❌ Ошибка сбора иерархических данных из {dataset_path}: {e}")
    
    return image_label_pairs

def create_debug_dataset_structure(train_datasets: List[str], limit: int = 800) -> str:
    """Создание временной структуры датасета для отладки с ограниченным количеством изображений"""
    if not IS_DEBUG:
        return None
    
    logger.info(f"🐛 Создание debug структуры с ограничением {limit} изображений")
    
    # Создание временных директорий
    debug_base_dir = '/kaggle/working/debug_dataset'
    debug_train_dir = os.path.join(debug_base_dir, 'train')
    debug_train_images = os.path.join(debug_train_dir, 'images')
    debug_train_labels = os.path.join(debug_train_dir, 'labels')
    
    # Очистка и создание директорий
    if os.path.exists(debug_base_dir):
        shutil.rmtree(debug_base_dir)
    
    os.makedirs(debug_train_images, exist_ok=True)
    os.makedirs(debug_train_labels, exist_ok=True)
    
    total_copied = 0
    
    # Копирование ограниченного количества файлов
    for dataset_path in train_datasets:
        if total_copied >= limit:
            break
            
        images_dir = os.path.join(dataset_path, 'images')
        labels_dir = os.path.join(dataset_path, 'labels')
        
        if not (os.path.exists(images_dir) and os.path.exists(labels_dir)):
            logger.warning(f"⚠️ Не найдены папки images/labels в {dataset_path}")
            continue
        
        # Проверка на иерархическую структуру (подпапки)
        try:
            subdirs = [d for d in os.listdir(images_dir) 
                      if os.path.isdir(os.path.join(images_dir, d))]
            
            if subdirs:  # Иерархическая структура (как TRAIN_DATASET_1)
                logger.info(f"📁 Обнаружена иерархическая структура в {dataset_path}")
                image_label_pairs = collect_hierarchical_images(dataset_path)
                
                # Ограничение количества пар для отладки
                selected_pairs = image_label_pairs[:limit - total_copied]
                
                for img_path, label_path in selected_pairs:
                    if total_copied >= limit:
                        break
                    
                    # Получение имени файла
                    img_filename = os.path.basename(img_path)
                    label_filename = os.path.basename(label_path)
                    
                    # Создание уникальных имен для отладочного датасета
                    dst_img = os.path.join(debug_train_images, f"{total_copied:06d}_{img_filename}")
                    dst_label = os.path.join(debug_train_labels, f"{total_copied:06d}_{label_filename}")
                    
                    # Копирование файлов
                    try:
                        shutil.copy2(img_path, dst_img)
                        shutil.copy2(label_path, dst_label)
                        total_copied += 1
                    except Exception as e:
                        logger.warning(f"⚠️ Ошибка копирования {img_path}: {e}")
                        continue
                        
            else:  # Плоская структура (старый код)
                logger.info(f"📄 Обнаружена плоская структура в {dataset_path}")
                
                # Получение списка изображений
                images = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                images = sorted(images)[:min(limit - total_copied, len(images))]
                
                for img_file in images:
                    if total_copied >= limit:
                        break
                        
                    # Копирование изображения
                    src_img = os.path.join(images_dir, img_file)
                    dst_img = os.path.join(debug_train_images, f"{total_copied:06d}_{img_file}")
                    
                    try:
                        shutil.copy2(src_img, dst_img)
                        
                        # Копирование соответствующей метки
                        label_file = os.path.splitext(img_file)[0] + '.txt'
                        src_label = os.path.join(labels_dir, label_file)
                        dst_label = os.path.join(debug_train_labels, f"{total_copied:06d}_{label_file}")
                        
                        if os.path.exists(src_label):
                            shutil.copy2(src_label, dst_label)
                        
                        total_copied += 1
                        
                    except Exception as e:
                        logger.warning(f"⚠️ Ошибка копирования {img_file}: {e}")
                        continue
                        
        except Exception as e:
            logger.error(f"❌ Ошибка обработки датасета {dataset_path}: {e}")
            continue
    
    logger.info(f"✅ Создана debug структура: {total_copied} изображений в {debug_train_dir}")
    return debug_train_dir

def prepare_debug_dataset(train_paths: List[str], limit: int = 800) -> List[str]:
    """Подготовка ограниченного датасета для отладки"""
    if not IS_DEBUG:
        return train_paths
    
    logger.info(f"🐛 Режим отладки: ограничение до {limit} изображений")
    
    # Создание временной структуры
    debug_dir = create_debug_dataset_structure(train_paths, limit)
    if debug_dir:
        return [debug_dir]
    
    # Fallback к старой логике если создание структуры не удалось
    debug_train_paths = []
    total_images = 0
    
    for train_path in train_paths:
        if isinstance(train_path, str):
            images_dir = train_path
        else:
            continue
            
        if os.path.exists(images_dir):
            images = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            images = sorted(images)[:min(limit - total_images, len(images))]
            
            if images:
                debug_train_paths.append(images_dir)
                total_images += len(images)
                logger.info(f"📊 Добавлено {len(images)} изображений из {images_dir}")
            
            if total_images >= limit:
                break
    
    logger.info(f"🎯 Итого для отладки: {total_images} изображений")
    return debug_train_paths

def get_dataset_statistics(dataset_path: str) -> Dict:
    """Получение статистики датасета с поддержкой иерархической структуры"""
    stats = {
        'images_count': 0,
        'labels_count': 0,
        'classes_distribution': {},
        'image_sizes': [],
        'exists': False,
        'structure_type': 'unknown'
    }
    
    if not os.path.exists(dataset_path):
        return stats
    
    stats['exists'] = True
    images_dir = os.path.join(dataset_path, 'images')
    labels_dir = os.path.join(dataset_path, 'labels')
    
    if not (os.path.exists(images_dir) and os.path.exists(labels_dir)):
        return stats
    
    # Проверка на иерархическую структуру
    try:
        subdirs = [d for d in os.listdir(images_dir) 
                  if os.path.isdir(os.path.join(images_dir, d))]
        
        if subdirs:  # Иерархическая структура
            stats['structure_type'] = 'hierarchical'
            logger.info(f"📁 Анализ иерархической структуры в {dataset_path}")
            
            # Использование функции collect_hierarchical_images для подсчета
            image_label_pairs = collect_hierarchical_images(dataset_path)
            stats['images_count'] = len(image_label_pairs)
            stats['labels_count'] = len(image_label_pairs)
            
            # Анализ классов из меток
            class_counts = {}
            for _, label_path in image_label_pairs:
                try:
                    with open(label_path, 'r') as f:
                        lines = f.readlines()
                        for line in lines:
                            if line.strip():
                                class_id = int(line.split()[0])
                                class_counts[class_id] = class_counts.get(class_id, 0) + 1
                except Exception as e:
                    continue
            
            stats['classes_distribution'] = class_counts
            
        else:  # Плоская структура
            stats['structure_type'] = 'flat'
            logger.info(f"📄 Анализ плоской структуры в {dataset_path}")
            
            # Подсчет изображений
            try:
                image_files = [f for f in os.listdir(images_dir) 
                              if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                stats['images_count'] = len(image_files)
                
                # Подсчет меток
                label_files = [f for f in os.listdir(labels_dir) 
                              if f.lower().endswith('.txt')]
                stats['labels_count'] = len(label_files)
                
                # Анализ классов
                class_counts = {}
                for label_file in label_files:
                    label_path = os.path.join(labels_dir, label_file)
                    try:
                        with open(label_path, 'r') as f:
                            lines = f.readlines()
                            for line in lines:
                                if line.strip():
                                    class_id = int(line.split()[0])
                                    class_counts[class_id] = class_counts.get(class_id, 0) + 1
                    except Exception as e:
                        continue
                
                stats['classes_distribution'] = class_counts
                
            except Exception as e:
                logger.error(f"❌ Ошибка анализа плоской структуры {dataset_path}: {e}")
                
    except Exception as e:
        logger.error(f"❌ Ошибка анализа структуры датасета {dataset_path}: {e}")
    
    return stats

def validate_on_private_dataset(model_path: str, private_dataset_path: str = VAL_DATASET_PRIVATE) -> Dict:
    """Валидация модели на приватном датасете"""
    logger.info("🔒 Начало валидации на приватном датасете...")
    
    if not os.path.exists(model_path):
        logger.error(f"❌ Модель не найдена: {model_path}")
        return {'error': 'Model not found'}
    
    if not os.path.exists(private_dataset_path):
        logger.error(f"❌ Приватный датасет не найден: {private_dataset_path}")
        return {'error': 'Private dataset not found'}
    
    try:
        # Загрузка модели
        model = YOLO(model_path)
        logger.info(f"✅ Модель загружена: {model_path}")
        
        # Создание временного YAML для приватного датасета
        private_yaml_config = {
            'path': os.path.dirname(private_dataset_path),
            'train': os.path.join(TRAIN_DATASET_1, 'images'),  # Добавляем обязательный 'train' ключ
            'val': os.path.join(private_dataset_path, 'images'),
            'nc': 1,
            'names': ['person']
        }
        
        private_yaml_path = 'private_validation.yaml'
        with open(private_yaml_path, 'w', encoding='utf-8') as f:
            yaml.dump(private_yaml_config, f, default_flow_style=False, allow_unicode=True)
        
        # Валидация
        logger.info("🔍 Выполнение валидации...")
        val_results = model.val(
            data=private_yaml_path,
            imgsz=640,
            batch=16,
            conf=0.25,
            iou=0.7,
            device='auto',
            plots=True,
            save_json=True
        )
        
        # Извлечение результатов
        results_dict = {
            'map50': float(val_results.box.map50) if hasattr(val_results, 'box') else 0.0,
            'map50_95': float(val_results.box.map) if hasattr(val_results, 'box') else 0.0,
            'precision': float(val_results.box.mp) if hasattr(val_results, 'box') else 0.0,
            'recall': float(val_results.box.mr) if hasattr(val_results, 'box') else 0.0,
            'f1_score': 0.0,
            'custom_metric_q': 0.0
        }
        
        # Вычисление F1-score
        if results_dict['precision'] > 0 and results_dict['recall'] > 0:
            results_dict['f1_score'] = 2 * (results_dict['precision'] * results_dict['recall']) / (results_dict['precision'] + results_dict['recall'])
        
        # Вычисление кастомной метрики Q
        logger.info("🎯 Вычисление кастомной метрики Q...")
        try:
            predictions, ground_truths = get_model_predictions_as_masks(model, private_dataset_path)
            if len(predictions) > 0 and len(ground_truths) > 0:
                custom_q = calculate_custom_metric(predictions, ground_truths, beta=1.0)
                results_dict['custom_metric_q'] = custom_q
            else:
                logger.warning("⚠️ Не удалось получить маски для вычисления кастомной метрики")
        except Exception as e:
            logger.error(f"❌ Ошибка вычисления кастомной метрики: {e}")
        
        logger.info("📊 Результаты валидации на приватном датасете:")
        logger.info(f"   mAP@0.5: {results_dict['map50']:.4f}")
        logger.info(f"   mAP@0.5:0.95: {results_dict['map50_95']:.4f}")
        logger.info(f"   Precision: {results_dict['precision']:.4f}")
        logger.info(f"   Recall: {results_dict['recall']:.4f}")
        logger.info(f"   F1-Score: {results_dict['f1_score']:.4f}")
        logger.info(f"   🎯 Кастомная метрика Q: {results_dict['custom_metric_q']:.4f}")
        
        # Очистка временного файла
        try:
            os.remove(private_yaml_path)
        except:
            pass
        
        return results_dict
        
    except Exception as e:
        logger.error(f"❌ Ошибка валидации на приватном датасете: {e}")
        import traceback
        traceback.print_exc()
        return {'error': str(e)}

def calculate_custom_metric(predictions: List[np.ndarray], ground_truths: List[np.ndarray], beta: float = 1.0) -> float:
    """
    Вычисление кастомной метрики согласно формуле (1)
    
    Args:
        predictions: Список предсказанных масок для каждого изображения
        ground_truths: Список истинных масок для каждого изображения  
        beta: Параметр для F-beta меры (по умолчанию 1.0)
        
    Returns:
        float: Значение кастомной метрики Q
    """
    # Пороговые значения от 0.3 до 0.93 с шагом 0.07
    thresholds = np.arange(0.3, 0.94, 0.07)
    n_thresholds = len(thresholds)
    
    logger.info(f"📊 Вычисление кастомной метрики для {n_thresholds} порогов: {thresholds}")
    
    total_f_beta = 0.0
    
    for threshold in thresholds:
        f_beta_t = calculate_f_beta_for_threshold(predictions, ground_truths, threshold, beta)
        total_f_beta += f_beta_t
        logger.info(f"   Порог {threshold:.2f}: F-beta = {f_beta_t:.4f}")
    
    # Финальная метрика - среднее арифметическое
    custom_metric = total_f_beta / n_thresholds
    
    logger.info(f"🎯 Итоговая кастомная метрика Q = {custom_metric:.4f}")
    return custom_metric


def calculate_f_beta_for_threshold(predictions: List[np.ndarray], ground_truths: List[np.ndarray], 
                                  threshold: float, beta: float = 1.0) -> float:
    """
    Вычисление F-beta меры для конкретного порога IoU
    
    Args:
        predictions: Список предсказанных масок
        ground_truths: Список истинных масок
        threshold: Пороговое значение IoU
        beta: Параметр для F-beta меры
        
    Returns:
        float: Значение F-beta меры для данного порога
    """
    total_tp = 0
    total_fp = 0
    total_fn = 0
    
    for pred_mask, gt_mask in zip(predictions, ground_truths):
        tp, fp, fn = calculate_tp_fp_fn_for_image(pred_mask, gt_mask, threshold)
        total_tp += tp
        total_fp += fp
        total_fn += fn
    
    # Вычисление F-beta меры по формуле (2)
    if total_tp + total_fp == 0 and total_tp + total_fn == 0:
        return 0.0
    
    if total_tp + total_fp == 0:
        precision = 0.0
    else:
        precision = total_tp / (total_tp + total_fp)
    
    if total_tp + total_fn == 0:
        recall = 0.0
    else:
        recall = total_tp / (total_tp + total_fn)
    
    if precision + recall == 0:
        return 0.0
    
    # F-beta формула с beta = 1
    f_beta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
    
    return f_beta


def calculate_tp_fp_fn_for_image(pred_mask: np.ndarray, gt_mask: np.ndarray, threshold: float) -> Tuple[int, int, int]:
    """
    Вычисление TP, FP, FN для одного изображения согласно алгоритму
    
    Args:
        pred_mask: Предсказанная маска (2D массив с 0 и 1)
        gt_mask: Истинная маска (2D массив с 0 и 1)
        threshold: Пороговое значение IoU
        
    Returns:
        Tuple[int, int, int]: TP, FP, FN
    """
    # Извлечение областей предсказания и разметки
    pred_regions = extract_regions(pred_mask)
    gt_regions = extract_regions(gt_mask)
    
    if len(pred_regions) == 0 and len(gt_regions) == 0:
        return 0, 0, 0
    
    if len(pred_regions) == 0:
        return 0, 0, len(gt_regions)
    
    if len(gt_regions) == 0:
        return 0, len(pred_regions), 0
    
    # Создание матрицы соответствия IoU
    iou_matrix = create_iou_matrix(pred_regions, gt_regions)
    
    # Подсчет TP, FP, FN согласно алгоритму
    tp, fp, fn = count_tp_fp_fn_from_matrix(iou_matrix, threshold)
    
    return tp, fp, fn


def extract_regions(mask: np.ndarray) -> List[np.ndarray]:
    """
    Извлечение связных областей из бинарной маски
    
    Args:
        mask: Бинарная маска (2D массив с 0 и 1)
        
    Returns:
        List[np.ndarray]: Список масок для каждой связной области
    """
    # Поиск связных компонент
    labeled_mask, num_features = ndimage.label(mask)
    
    regions = []
    for i in range(1, num_features + 1):
        region_mask = (labeled_mask == i).astype(np.uint8)
        regions.append(region_mask)
    
    return regions


def get_model_predictions_as_masks(model, dataset_path: str, img_size: int = 640, conf_threshold: float = 0.25) -> Tuple[List[np.ndarray], List[np.ndarray]]:
    """
    Получение предсказаний модели в виде бинарных масок
    
    Args:
        model: Обученная модель YOLO
        dataset_path: Путь к датасету
        img_size: Размер изображения для инференса
        conf_threshold: Порог уверенности для детекций
        
    Returns:
        Tuple[List[np.ndarray], List[np.ndarray]]: Предсказанные маски и истинные маски
    """
    images_dir = os.path.join(dataset_path, 'images')
    labels_dir = os.path.join(dataset_path, 'labels')
    
    if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
        logger.error(f"❌ Не найдены папки images или labels в {dataset_path}")
        return [], []
    
    # Проверяем структуру датасета
    subdirs = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
    
    if subdirs:  # Иерархическая структура
        logger.info(f"📁 Обнаружена иерархическая структура датасета с {len(subdirs)} подпапками")
        image_label_pairs = collect_hierarchical_images(dataset_path)
        
        if not image_label_pairs:
            logger.error(f"❌ Не найдено пар изображение-разметка в {dataset_path}")
            return [], []
            
        predictions = []
        ground_truths = []
        
        logger.info(f"🔍 Обработка {len(image_label_pairs)} пар изображение-разметка для получения масок...")
        
        for i, (img_path, label_path) in enumerate(image_label_pairs):
            if i % 50 == 0:
                logger.info(f"   Обработано {i}/{len(image_label_pairs)} изображений")
            
            # Загрузка изображения
            try:
                image = Image.open(img_path)
                img_width, img_height = image.size
            except Exception as e:
                logger.warning(f"⚠️ Ошибка загрузки изображения {img_path}: {e}")
                continue
            
            # Получение предсказания модели
            try:
                results = model.predict(img_path, imgsz=img_size, conf=conf_threshold, verbose=False)
                pred_mask = create_mask_from_yolo_results(results[0], img_width, img_height)
            except Exception as e:
                logger.warning(f"⚠️ Ошибка предсказания для {img_path}: {e}")
                pred_mask = np.zeros((img_height, img_width), dtype=np.uint8)
            
            # Загрузка истинной разметки
            try:
                gt_mask = create_mask_from_yolo_labels(label_path, img_width, img_height)
            except Exception as e:
                logger.warning(f"⚠️ Ошибка загрузки разметки для {label_path}: {e}")
                gt_mask = np.zeros((img_height, img_width), dtype=np.uint8)
            
            predictions.append(pred_mask)
            ground_truths.append(gt_mask)
            
    else:  # Плоская структура
        logger.info(f"📄 Обнаружена плоская структура датасета")
        image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        predictions = []
        ground_truths = []
        
        logger.info(f"🔍 Обработка {len(image_files)} изображений для получения масок...")
        
        for i, img_file in enumerate(image_files):
            if i % 50 == 0:
                logger.info(f"   Обработано {i}/{len(image_files)} изображений")
                
            img_path = os.path.join(images_dir, img_file)
            label_path = os.path.join(labels_dir, img_file.rsplit('.', 1)[0] + '.txt')
            
            # Загрузка изображения
            try:
                image = Image.open(img_path)
                img_width, img_height = image.size
            except Exception as e:
                logger.warning(f"⚠️ Ошибка загрузки изображения {img_file}: {e}")
                continue
            
            # Получение предсказания модели
            try:
                results = model.predict(img_path, imgsz=img_size, conf=conf_threshold, verbose=False)
                pred_mask = create_mask_from_yolo_results(results[0], img_width, img_height)
            except Exception as e:
                logger.warning(f"⚠️ Ошибка предсказания для {img_file}: {e}")
                pred_mask = np.zeros((img_height, img_width), dtype=np.uint8)
            
            # Загрузка истинной разметки
            try:
                gt_mask = create_mask_from_yolo_labels(label_path, img_width, img_height)
            except Exception as e:
                logger.warning(f"⚠️ Ошибка загрузки разметки для {img_file}: {e}")
                gt_mask = np.zeros((img_height, img_width), dtype=np.uint8)
            
            predictions.append(pred_mask)
            ground_truths.append(gt_mask)
    
    logger.info(f"✅ Получено {len(predictions)} пар масок для оценки")
    return predictions, ground_truths


def create_mask_from_yolo_results(results, img_width: int, img_height: int) -> np.ndarray:
    """
    Создание бинарной маски из результатов YOLO
    
    Args:
        results: Результаты детекции YOLO
        img_width: Ширина изображения
        img_height: Высота изображения
        
    Returns:
        np.ndarray: Бинарная маска
    """
    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    
    if results.boxes is not None and len(results.boxes) > 0:
        boxes = results.boxes.xyxy.cpu().numpy()  # Координаты bbox в формате x1,y1,x2,y2
        
        for box in boxes:
            x1, y1, x2, y2 = map(int, box[:4])
            # Ограничиваем координаты размерами изображения
            x1 = max(0, min(x1, img_width-1))
            y1 = max(0, min(y1, img_height-1))
            x2 = max(0, min(x2, img_width-1))
            y2 = max(0, min(y2, img_height-1))
            
            # Заполняем область bbox единицами
            mask[y1:y2+1, x1:x2+1] = 1
    
    return mask


def create_mask_from_yolo_labels(label_path: str, img_width: int, img_height: int) -> np.ndarray:
    """
    Создание бинарной маски из YOLO разметки
    
    Args:
        label_path: Путь к файлу разметки
        img_width: Ширина изображения
        img_height: Высота изображения
        
    Returns:
        np.ndarray: Бинарная маска
    """
    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    
    if not os.path.exists(label_path):
        return mask
    
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                # YOLO формат: class_id center_x center_y width height (нормализованные координаты)
                center_x = float(parts[1]) * img_width
                center_y = float(parts[2]) * img_height
                width = float(parts[3]) * img_width
                height = float(parts[4]) * img_height
                
                # Преобразование в координаты bbox
                x1 = int(center_x - width/2)
                y1 = int(center_y - height/2)
                x2 = int(center_x + width/2)
                y2 = int(center_y + height/2)
                
                # Ограничиваем координаты размерами изображения
                x1 = max(0, min(x1, img_width-1))
                y1 = max(0, min(y1, img_height-1))
                x2 = max(0, min(x2, img_width-1))
                y2 = max(0, min(y2, img_height-1))
                
                # Заполняем область bbox единицами
                mask[y1:y2+1, x1:x2+1] = 1
    
    except Exception as e:
        logger.warning(f"⚠️ Ошибка чтения разметки {label_path}: {e}")
    
    return mask


def calculate_iou(region_a: np.ndarray, region_b: np.ndarray) -> float:
    """
    Вычисление IoU между двумя областями по формуле (3)
    
    Args:
        region_a: Первая область (бинарная маска)
        region_b: Вторая область (бинарная маска)
        
    Returns:
        float: Значение IoU
    """
    intersection = np.logical_and(region_a, region_b).sum()
    union = np.logical_or(region_a, region_b).sum()
    
    if union == 0:
        return 0.0
    
    return intersection / union


def create_iou_matrix(pred_regions: List[np.ndarray], gt_regions: List[np.ndarray]) -> np.ndarray:
    """
    Создание матрицы IoU между предсказанными и истинными областями
    
    Args:
        pred_regions: Список предсказанных областей
        gt_regions: Список истинных областей
        
    Returns:
        np.ndarray: Матрица IoU размером [len(pred_regions), len(gt_regions)]
    """
    iou_matrix = np.zeros((len(pred_regions), len(gt_regions)))
    
    for i, pred_region in enumerate(pred_regions):
        for j, gt_region in enumerate(gt_regions):
            iou_matrix[i, j] = calculate_iou(pred_region, gt_region)
    
    return iou_matrix


def count_tp_fp_fn_from_matrix(iou_matrix: np.ndarray, threshold: float) -> Tuple[int, int, int]:
    """
    Подсчет TP, FP, FN из матрицы IoU согласно описанному алгоритму
    
    Args:
        iou_matrix: Матрица IoU
        threshold: Пороговое значение
        
    Returns:
        Tuple[int, int, int]: TP, FP, FN
    """
    tp = 0
    matrix_copy = iou_matrix.copy()
    
    while matrix_copy.size > 0:
        # Поиск максимального элемента
        max_val = np.max(matrix_copy)
        
        if max_val >= threshold:
            # Увеличиваем TP
            tp += 1
            
            # Находим позицию максимального элемента
            max_pos = np.unravel_index(np.argmax(matrix_copy), matrix_copy.shape)
            row_idx, col_idx = max_pos
            
            # Удаляем строку и столбец
            matrix_copy = np.delete(matrix_copy, row_idx, axis=0)
            matrix_copy = np.delete(matrix_copy, col_idx, axis=1)
        else:
            # Прерываем процедуру
            break
    
    # FN = количество оставшихся столбцов (истинных объектов)
    fn = matrix_copy.shape[1] if matrix_copy.size > 0 else 0
    
    # FP = количество оставшихся строк (предсказанных объектов)
    fp = matrix_copy.shape[0] if matrix_copy.size > 0 else 0
    
    return tp, fp, fn


def cleanup_debug_files():
    """Очистка временных debug файлов"""
    debug_base_dir = '/kaggle/working'
    
    if os.path.exists(debug_base_dir):
        try:
            shutil.rmtree(debug_base_dir)
            logger.info("🧹 Временные debug файлы очищены")
        except Exception as e:
            logger.warning(f"⚠️ Ошибка очистки debug файлов: {e}")
    
    # Очистка временных YAML файлов
    temp_yaml_files = ['kaggle_combined_data.yaml', 'combined_data.yaml', 'private_validation.yaml']
    for yaml_file in temp_yaml_files:
        if os.path.exists(yaml_file):
            try:
                os.remove(yaml_file)
            except:
                pass

def analyze_all_datasets() -> Dict:
    """Анализ всех доступных датасетов"""
    logger.info("📊 Анализ всех датасетов...")
    
    datasets = {
        'train_dataset_1': get_dataset_statistics(TRAIN_DATASET_1),
        'train_dataset_2': get_dataset_statistics(TRAIN_DATASET_2),
        'val_dataset_public': get_dataset_statistics(VAL_DATASET_PUBLIC),
        'val_dataset_private': get_dataset_statistics(VAL_DATASET_PRIVATE)
    }
    
    total_train_images = 0
    total_train_labels = 0
    
    for name, stats in datasets.items():
        if stats['exists']:
            structure_info = f" ({stats['structure_type']})" if 'structure_type' in stats else ""
            logger.info(f"✅ {name}{structure_info}: {stats['images_count']} изображений, {stats['labels_count']} меток")
            if 'train' in name:
                total_train_images += stats['images_count']
                total_train_labels += stats['labels_count']
            if stats['classes_distribution']:
                logger.info(f"   Распределение классов: {stats['classes_distribution']}")
        else:
            logger.warning(f"❌ {name}: датасет не найден")
    
    logger.info(f"🎯 Итого для обучения: {total_train_images} изображений, {total_train_labels} меток")
    
    return datasets

class AdvancedDynamicRoutingModule(nn.Module):
    """Продвинутый модуль динамической маршрутизации с многоуровневой адаптацией для YOLOv13"""
    
    def __init__(self, in_channels: int, out_channels: int, num_routes: int = 3, 
                 routing_iterations: int = 2, capsule_dim: int = 8):
        super().__init__()
        self.num_routes = num_routes
        self.routing_iterations = routing_iterations
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.capsule_dim = capsule_dim
        
        # Упрощенные капсулы для эффективности
        self.primary_capsules = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels // num_routes, 3, padding=1),
                nn.BatchNorm2d(out_channels // num_routes),
                nn.SiLU()
            ) for _ in range(num_routes)
        ])
        
        # Адаптивные весовые матрицы
        self.routing_weights = nn.Parameter(torch.randn(num_routes, capsule_dim, capsule_dim) * 0.1)
        self.temperature = nn.Parameter(torch.ones(1))
        
        # UAV-специфичные адаптации
        self.altitude_encoder = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels // num_routes, capsule_dim, 1),
            nn.SiLU()
        )
        
        # Объединение маршрутов
        self.route_fusion = nn.Conv2d(out_channels, out_channels, 1)
        self.norm = nn.BatchNorm2d(out_channels)
        
    def enhanced_squash(self, tensor: torch.Tensor, dim: int = -1, epsilon: float = 1e-8) -> torch.Tensor:
        """Улучшенная функция сжатия"""
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm + epsilon)
        unit_vector = tensor / torch.sqrt(squared_norm + epsilon)
        return scale * unit_vector
    
    def dynamic_routing(self, route_features: List[torch.Tensor]) -> torch.Tensor:
        """Динамическая маршрутизация между путями"""
        batch_size = route_features[0].size(0)
        device = route_features[0].device
        
        # Инициализация весов маршрутизации
        routing_logits = torch.zeros(batch_size, self.num_routes, device=device)
        
        for iteration in range(self.routing_iterations):
            # Softmax для получения весов
            routing_weights = F.softmax(routing_logits / self.temperature, dim=1)
            
            # Взвешенное объединение
            weighted_features = []
            for i, features in enumerate(route_features):
                weight = routing_weights[:, i:i+1, None, None]
                weighted_features.append(features * weight)
            
            combined = torch.stack(weighted_features, dim=1).sum(dim=1)
            
            if iteration < self.routing_iterations - 1:
                # Обновление логитов на основе согласованности
                for i, features in enumerate(route_features):
                    agreement = F.cosine_similarity(
                        features.flatten(1), combined.flatten(1), dim=1
                    )
                    routing_logits[:, i] += agreement
        
        return combined
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Обработка через различные маршруты
        route_outputs = []
        for capsule in self.primary_capsules:
            route_out = capsule(x)
            route_outputs.append(route_out)
        
        # Динамическая маршрутизация
        routed_output = self.dynamic_routing(route_outputs)
        
        # Объединение всех маршрутов
        all_routes = torch.cat(route_outputs, dim=1)
        fused = self.route_fusion(all_routes)
        
        # Остаточное соединение с маршрутизированным выходом
        if routed_output.size(1) == fused.size(1):
            output = fused + 0.3 * routed_output
        else:
            output = fused
        
        return self.norm(output)

class EnhancedUAVAdaptiveBlock(nn.Module):
    """Улучшенный адаптивный блок для UAV с многофакторной адаптацией"""
    
    def __init__(self, in_channels: int, out_channels: int, reduction_ratio: int = 16):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        # Основные конволюционные слои
        self.conv1 = Conv(in_channels, out_channels, 3, 1)
        self.conv2 = Conv(out_channels, out_channels, 3, 1)
        
        # Канальное внимание (упрощенное)
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels, max(out_channels // reduction_ratio, 1), 1),
            nn.SiLU(),
            nn.Conv2d(max(out_channels // reduction_ratio, 1), out_channels, 1),
            nn.Sigmoid()
        )
        
        # Пространственное внимание
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, 7, padding=3),
            nn.Sigmoid()
        )
        
        # UAV-специфичные адаптации
        self.altitude_adaptation = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(out_channels, out_channels, 1),
            nn.Sigmoid()
        )
        
        # Проекция для остаточного соединения
        self.shortcut = nn.Identity() if in_channels == out_channels else Conv(in_channels, out_channels, 1, 1)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = self.shortcut(x)
        
        # Основная обработка
        out = self.conv1(x)
        out = self.conv2(out)
        
        # Канальное внимание
        ca = self.channel_attention(out)
        out = out * ca
        
        # Пространственное внимание
        avg_pool = torch.mean(out, dim=1, keepdim=True)
        max_pool, _ = torch.max(out, dim=1, keepdim=True)
        spatial_input = torch.cat([avg_pool, max_pool], dim=1)
        sa = self.spatial_attention(spatial_input)
        out = out * sa
        
        # UAV адаптация
        altitude_att = self.altitude_adaptation(out)
        out = out * altitude_att
        
        # Остаточное соединение
        return out + residual

class YOLOv13DynamicUAV(nn.Module):
    """YOLOv13 с динамической маршрутизацией для UAV"""
    
    def __init__(self, base_model, num_classes=1):
        super().__init__()
        self.base_model = base_model
        self.num_classes = num_classes
        
        # Интеграция динамических модулей в backbone
        self.dynamic_modules = nn.ModuleList([
            AdvancedDynamicRoutingModule(64, 64),
            AdvancedDynamicRoutingModule(128, 128),
            AdvancedDynamicRoutingModule(256, 256),
        ])
        
        # UAV адаптивные блоки
        self.uav_blocks = nn.ModuleList([
            EnhancedUAVAdaptiveBlock(64, 64),
            EnhancedUAVAdaptiveBlock(128, 128),
            EnhancedUAVAdaptiveBlock(256, 256),
        ])
        
    def forward(self, x):
        # Используем базовую YOLO модель с интеграцией наших модулей
        return self.base_model(x)

def optimize_gpu_memory():
    """Оптимизация GPU памяти для Kaggle"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        # Установка фракции памяти для избежания OOM
        torch.cuda.set_per_process_memory_fraction(0.8)
        logger.info("🚀 GPU память оптимизирована")

def detect_device_kaggle():
    """Улучшенное определение устройства для Kaggle"""
    # Проверка переменных окружения Kaggle
    is_kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None
    kaggle_accelerator = os.environ.get('KAGGLE_ACCELERATOR', 'None')
    
    if is_kaggle:
        logger.info(f"🏃 Запуск в Kaggle, ускоритель: {kaggle_accelerator}")
    
    # Проверка доступности CUDA
    if torch.cuda.is_available() and torch.cuda.device_count() > 0:
        try:
            # Тестирование GPU
            device = torch.device('cuda:0')
            test_tensor = torch.randn(100, 100, device=device)
            _ = test_tensor @ test_tensor.T
            
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            
            logger.info(f"🔥 Используется GPU: {gpu_name}")
            logger.info(f"💾 GPU память: {gpu_memory:.2f} GB")
            
            # Очистка
            del test_tensor
            torch.cuda.empty_cache()
            
            # Оптимизация GPU
            optimize_gpu_memory()
            
            return 'cuda:0'
            
        except Exception as e:
            logger.warning(f"⚠️ GPU доступен, но не работает: {e}")
            return 'cpu'
    else:
        logger.info("💻 Используется CPU (GPU недоступен)")
        return 'cpu'

def get_optimal_config_for_device(device: str) -> dict:
    """Получение оптимальной конфигурации для устройства"""
    if device.startswith('cuda'):
        return {
            'batch_size': 16,
            'workers': 4,
            'mixed_precision': True,
            'cache': True,
            'optimizer': 'AdamW',
            'lr0': 0.001,
            'epochs': 100
        }
    else:
        return {
            'batch_size': 4,
            'workers': 2,
            'mixed_precision': False,
            'cache': False,
            'optimizer': 'SGD',
            'lr0': 0.01,
            'epochs': 50
        }

def create_yolov13_config(num_classes: int = 1) -> dict:
    """Создание конфигурации YOLOv13 с динамической маршрутизацией"""
    return {
        'nc': num_classes,
        'depth_multiple': 0.67,
        'width_multiple': 0.75,
        'anchors': 3,
        
        # Backbone с интеграцией динамических модулей
        'backbone': [
            [-1, 1, 'Conv', [64, 6, 2, 2]],  # 0-P1/2
            [-1, 1, 'Conv', [128, 3, 2]],    # 1-P2/4
            [-1, 3, 'C2f', [128, True]],     # 2
            [-1, 1, 'Conv', [256, 3, 2]],    # 3-P3/8
            [-1, 6, 'C2f', [256, True]],     # 4
            [-1, 1, 'Conv', [512, 3, 2]],    # 5-P4/16
            [-1, 6, 'C2f', [512, True]],     # 6
            [-1, 1, 'Conv', [1024, 3, 2]],   # 7-P5/32
            [-1, 3, 'C2f', [1024, True]],    # 8
            [-1, 1, 'SPPF', [1024, 5]],      # 9
        ],
        
        # Head с улучшенной архитектурой для UAV
        'head': [
            [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],  # 10
            [[-1, 6], 1, 'Concat', [1]],                    # 11
            [-1, 3, 'C2f', [512]],                          # 12
            [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],   # 13
            [[-1, 4], 1, 'Concat', [1]],                    # 14
            [-1, 3, 'C2f', [256]],                          # 15
            [-1, 1, 'Conv', [256, 3, 2]],                   # 16
            [[-1, 12], 1, 'Concat', [1]],                   # 17
            [-1, 3, 'C2f', [512]],                          # 18
            [-1, 1, 'Conv', [512, 3, 2]],                   # 19
            [[-1, 9], 1, 'Concat', [1]],                    # 20
            [-1, 3, 'C2f', [1024]],                         # 21
            [[15, 18, 21], 1, 'Detect', [num_classes]],     # 22 Detect
        ]
    }

def train_yolov13_dynamic_uav(
    data_yaml: str = None,
    epochs: int = 100,
    batch_size: int = 16,
    img_size: int = 640,
    device: str = 'auto',
    project: str = 'yolov13_uav_runs',
    name: str = 'dynamic_routing_experiment',
    save_dir: str = './models',
    use_kaggle_datasets: bool = True,
    **kwargs
) -> Tuple[object, Dict]:
    """Обучение YOLOv13 с динамической маршрутизацией для UAV
    
    Args:
        data_yaml: Путь к YAML файлу с конфигурацией датасета (если None, используются Kaggle датасеты)
        epochs: Количество эпох обучения
        batch_size: Размер батча
        img_size: Размер входных изображений
        device: Устройство для обучения ('auto', 'cpu', 'cuda')
        project: Папка проекта для сохранения результатов
        name: Имя эксперимента
        save_dir: Директория для сохранения моделей
        use_kaggle_datasets: Использовать Kaggle датасеты вместо data_yaml
        **kwargs: Дополнительные параметры
    
    Returns:
        Tuple[object, Dict]: Обученная модель и результаты обучения
    """
    
    # Автоматическое определение устройства
    if device == 'auto':
        device = detect_device_kaggle()
    
    # Получение оптимальной конфигурации
    device_config = get_optimal_config_for_device(device)
    
    # Адаптация параметров
    batch_size = min(batch_size, device_config['batch_size'])
    epochs = min(epochs, device_config['epochs'])
    
    # Создание директорий
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(project, exist_ok=True)
    
    logger.info("🚁 YOLOv13 Dynamic UAV - Система обнаружения людей для БПЛА")
    logger.info("=" * 65)
    
    # Обработка режима отладки
    if IS_DEBUG:
        epochs = 1
        logger.info("🐛 РЕЖИМ ОТЛАДКИ АКТИВЕН")
        logger.info(f"   Эпохи: {epochs} (принудительно установлено для отладки)")
        logger.info(f"   Ограничение изображений: 800")
        logger.info(f"   Валидация: на всех данных")
    
    # Подготовка датасета
    if use_kaggle_datasets or data_yaml is None:
        logger.info("📊 Использование Kaggle датасетов")
        
        # Анализ доступных датасетов
        datasets_analysis = analyze_all_datasets()
        
        # Создание YAML конфигурации для объединенных датасетов
        yaml_path = create_combined_dataset_yaml('kaggle_combined_data.yaml')
        if not yaml_path:
            raise ValueError("❌ Не удалось создать YAML конфигурацию для датасетов")
        
        data_yaml = yaml_path
        logger.info(f"✅ Создана конфигурация датасета: {data_yaml}")
    
    logger.info(f"📊 Параметры обучения:")
    logger.info(f"   Датасет: {data_yaml}")
    logger.info(f"   Эпохи: {epochs}")
    logger.info(f"   Размер батча: {batch_size}")
    logger.info(f"   Размер изображения: {img_size}")
    logger.info(f"   Устройство: {device}")
    logger.info(f"   Режим отладки: {IS_DEBUG}")
    
    # Создание директории для кэша
    cache_dir = '/kaggle/working/cache'
    os.makedirs(cache_dir, exist_ok=True)
    logger.info(f"💽 Директория кэша: {cache_dir}")
    
    # Оптимизированная конфигурация для UAV
    training_config = {
        'data': data_yaml,
        'epochs': epochs,
        'batch': batch_size,
        'imgsz': img_size,
        'device': device,
        'project': project,
        'name': name,
        'save': True,
        'save_period': 5,
        'patience': 20,
        'workers': device_config['workers'],
        'cache': 'disk',  # Использование дискового кэша
        
        # Оптимизатор
        'optimizer': device_config['optimizer'],
        'lr0': device_config['lr0'],
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'cos_lr': True,
        
        # UAV-оптимизированные аугментации
        'hsv_h': 0.015,
        'hsv_s': 0.7,
        'hsv_v': 0.4,
        'degrees': 15.0,
        'translate': 0.1,
        'scale': 0.5,
        'shear': 2.0,
        'perspective': 0.0,
        'flipud': 0.0,
        'fliplr': 0.5,
        'mosaic': 1.0,
        'mixup': 0.15,
        'copy_paste': 0.3,
        
        # Оптимизированные веса потерь для людей
        'box': 7.5,
        'cls': 0.5,
        'dfl': 1.5,
        
        # NMS параметры
        'iou': 0.7,
        'conf': 0.25,
        
        # Валидация
        'val': True,
        'plots': True,
        'rect': False,
        
        **kwargs
    }
    
    try:
        # Инициализация базовой YOLO модели
        logger.info("🔄 Загрузка базовой YOLO модели...")
        base_model = YOLO('yolo12n.pt')
        
        # Создание конфигурации YOLOv13
        yolov13_config = create_yolov13_config(num_classes=1)
        
        # Сохранение конфигурации
        config_path = os.path.join(save_dir, 'yolov13_dynamic_uav.yaml')
        with open(config_path, 'w') as f:
            yaml.dump(yolov13_config, f, default_flow_style=False)
        
        logger.info(f"💾 Конфигурация YOLOv13 сохранена: {config_path}")
        
        # Обучение модели
        logger.info("🚀 Начало обучения YOLOv13 Dynamic UAV...")
        start_time = time.time()
        
        results = base_model.train(**training_config)
        
        training_time = time.time() - start_time
        logger.info(f"⏱️ Время обучения: {training_time/60:.2f} минут")
        
        # Сохранение лучшей модели
        best_model_path = os.path.join(save_dir, 'yolov13_dynamic_uav_best.pt')
        
        if hasattr(results, 'save_dir') and results.save_dir:
            weights_dir = Path(results.save_dir) / 'weights'
            best_path = weights_dir / 'best.pt'
            
            if best_path.exists():
                shutil.copy2(best_path, best_model_path)
                logger.info(f"✅ Лучшая модель сохранена: {best_model_path}")
            else:
                logger.warning("⚠️ Файл best.pt не найден")
        
        # Валидация модели
        logger.info("📊 Валидация модели...")
        try:
            val_results = base_model.val()
            if hasattr(val_results, 'box'):
                logger.info(f"📈 mAP50: {val_results.box.map50:.4f}")
                logger.info(f"📈 mAP50-95: {val_results.box.map:.4f}")
        except Exception as e:
            logger.warning(f"⚠️ Ошибка валидации: {e}")
            val_results = None
        
        # Валидация на приватном датасете (если доступен)
        if os.path.exists(best_model_path):
            logger.info("🔒 Валидация на приватном датасете...")
            private_val_results = validate_on_private_dataset(best_model_path)
            if 'error' not in private_val_results:
                logger.info("✅ Приватная валидация завершена успешно")
            else:
                logger.warning(f"⚠️ Ошибка приватной валидации: {private_val_results['error']}")
        
        # Экспорт модели
        logger.info("📦 Экспорт модели...")
        export_formats = ['onnx']
        if device.startswith('cuda'):
            export_formats.append('torchscript')
        
        for fmt in export_formats:
            try:
                export_path = base_model.export(
                    format=fmt,
                    imgsz=img_size,
                    optimize=True,
                    half=device.startswith('cuda')
                )
                logger.info(f"✅ Модель экспортирована в {fmt}: {export_path}")
            except Exception as e:
                logger.warning(f"⚠️ Ошибка экспорта в {fmt}: {e}")
        
        # Создание отчета
        report = {
            'model_path': best_model_path,
            'config_path': config_path,
            'training_config': training_config,
            'training_time_minutes': training_time / 60,
            'results': val_results.results_dict if val_results and hasattr(val_results, 'results_dict') else {},
            'training_completed': True,
            'device_used': device,
            'final_epochs': epochs,
            'final_batch_size': batch_size,
            'yolov13_features': {
                'dynamic_routing': True,
                'uav_optimization': True,
                'human_detection': True,
                'multi_scale_features': True
            }
        }
        
        logger.info("\n🎉 Обучение YOLOv13 Dynamic UAV завершено успешно!")
        logger.info(f"💾 Модель сохранена: {best_model_path}")
        logger.info(f"🎯 Модель готова для обнаружения людей с UAV!")
        
        return base_model, report
        
    except Exception as e:
        logger.error(f"❌ Ошибка во время обучения: {e}")
        import traceback
        traceback.print_exc()
        
        return None, {
            'config': training_config,
            'error': str(e),
            'training_completed': False,
            'device_used': device
        }

def prepare_uav_dataset(dataset_path: str, output_path: str) -> str:
    """Подготовка датасета для UAV обнаружения людей"""
    
    # Создание структуры директорий
    dirs = {
        'train_img': os.path.join(output_path, "train/images"),
        'train_lbl': os.path.join(output_path, "train/labels"),
        'val_img': os.path.join(output_path, "val/images"),
        'val_lbl': os.path.join(output_path, "val/labels")
    }
    
    for dir_path in dirs.values():
        os.makedirs(dir_path, exist_ok=True)
    
    # Проверка существования датасета
    images_path = os.path.join(dataset_path, "images")
    labels_path = os.path.join(dataset_path, "labels")
    
    if not os.path.exists(images_path):
        logger.warning(f"⚠️ Директория изображений не найдена: {images_path}")
        # Создание тестового data.yaml
        return create_test_data_yaml(output_path)
    
    # Получение списка изображений
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    image_files = [f for f in os.listdir(images_path) 
                  if f.lower().endswith(image_extensions)]
    
    if len(image_files) == 0:
        logger.warning("⚠️ Изображения не найдены")
        return create_test_data_yaml(output_path)
    
    # Разделение на train/val (80/20)
    train_files, val_files = train_test_split(
        image_files, test_size=0.2, random_state=42
    )
    
    def copy_dataset_files(files, split_type):
        copied_count = 0
        for file in files:
            # Копирование изображения
            img_src = os.path.join(images_path, file)
            img_dst = os.path.join(dirs[f'{split_type}_img'], file)
            
            if os.path.exists(img_src):
                shutil.copy2(img_src, img_dst)
                copied_count += 1
                
                # Копирование соответствующего файла меток
                label_file = os.path.splitext(file)[0] + '.txt'
                label_src = os.path.join(labels_path, label_file)
                label_dst = os.path.join(dirs[f'{split_type}_lbl'], label_file)
                
                if os.path.exists(label_src):
                    # Проверка и корректировка меток для класса "человек"
                    with open(label_src, 'r') as f:
                        lines = f.readlines()
                    
                    corrected_lines = []
                    for line in lines:
                        parts = line.strip().split()
                        if len(parts) >= 5:
                            # Изменяем класс на 0 (человек) если это необходимо
                            parts[0] = '0'
                            corrected_lines.append(' '.join(parts) + '\n')
                    
                    with open(label_dst, 'w') as f:
                        f.writelines(corrected_lines)
                else:
                    # Создание пустого файла меток
                    with open(label_dst, 'w') as f:
                        pass
        
        return copied_count
    
    train_count = copy_dataset_files(train_files, "train")
    val_count = copy_dataset_files(val_files, "val")
    
    # Создание data.yaml
    data_yaml_content = f"""# YOLOv13 Dynamic UAV Dataset Configuration
# Optimized for aerial human detection from UAV imagery

path: {output_path}
train: train/images
val: val/images

nc: 1
names: ['person']

# Dataset Statistics
train_images: {train_count}
val_images: {val_count}
total_images: {train_count + val_count}

# UAV Specific Settings
altitude_range: "10-500m"
optimal_detection_height: "50-150m"
min_object_size: "32x32 pixels"
recommended_confidence: 0.25
recommended_iou: 0.7
"""
    
    data_yaml_path = os.path.join(output_path, 'data.yaml')
    with open(data_yaml_path, 'w') as file:
        file.write(data_yaml_content)
    
    logger.info(f"✅ Датасет подготовлен: {train_count} train, {val_count} val")
    logger.info(f"📁 Конфигурация датасета: {data_yaml_path}")
    
    return data_yaml_path

def create_test_data_yaml(output_path: str) -> str:
    """Создание тестового data.yaml файла"""
    data_yaml_content = f"""# YOLOv13 Dynamic UAV Test Configuration
path: {output_path}
train: train/images
val: val/images

nc: 1
names: ['person']

# Test configuration - no actual dataset
test_mode: true
"""
    
    data_yaml_path = os.path.join(output_path, 'data.yaml')
    with open(data_yaml_path, 'w') as file:
        file.write(data_yaml_content)
    
    logger.info(f"📝 Создан тестовый data.yaml: {data_yaml_path}")
    return data_yaml_path

def main():
    """Основная функция для запуска обучения YOLOv13 с Kaggle датасетами"""
    logger.info("🚁 YOLOv13 Dynamic UAV - Система обнаружения людей для БПЛА")
    logger.info("=" * 70)
    # Очистка временных debug файлов
    if IS_DEBUG:
        cleanup_debug_files()
    
    # Проверка режима отладки
    if IS_DEBUG:
        logger.info("🐛 РЕЖИМ ОТЛАДКИ АКТИВЕН")
        logger.info("   • Обучение на 800 изображениях")
        logger.info("   • 1 эпоха обучения")
        logger.info("   • Валидация на всех данных")
    else:
        logger.info("🎯 ПОЛНЫЙ РЕЖИМ ОБУЧЕНИЯ")
        logger.info("   • Обучение на всех доступных данных")
        logger.info("   • Стандартное количество эпох")
    
    # Создание директорий
    data_path = Path("./data")
    models_path = Path("./models")
    
    data_path.mkdir(exist_ok=True)
    models_path.mkdir(exist_ok=True)
    
    # Анализ доступных Kaggle датасетов
    logger.info("\n📊 Анализ доступных Kaggle датасетов...")
    datasets_analysis = analyze_all_datasets()
    
    # Проверка доступности датасетов
    available_datasets = sum(1 for stats in datasets_analysis.values() if stats['exists'])
    if available_datasets == 0:
        logger.error("❌ Не найдено ни одного Kaggle датасета!")
        logger.error("   Проверьте пути к датасетам:")
        logger.error(f"   • {TRAIN_DATASET_1}")
        logger.error(f"   • {TRAIN_DATASET_2}")
        logger.error(f"   • {VAL_DATASET_PUBLIC}")
        logger.error(f"   • {VAL_DATASET_PRIVATE}")
        return None, {'error': 'Датасеты не найдены'}
    
    logger.info(f"✅ Найдено {available_datasets} из 4 датасетов")
    
    # Параметры обучения
    training_params = {
        'data_yaml': None,  # Будет создан автоматически из Kaggle датасетов
        'epochs': 1 if IS_DEBUG else 50,
        'batch_size': 8 if IS_DEBUG else 16,
        'img_size': 640,
        'device': 'auto',
        'save_dir': models_path,
        'project': 'yolov13_uav_human_detection',
        'name': f'kaggle_training_debug_{int(time.time())}' if IS_DEBUG else f'kaggle_training_{int(time.time())}',
        'use_kaggle_datasets': True
    }
    
    logger.info(f"\n📋 Параметры обучения:")
    for key, value in training_params.items():
        logger.info(f"   {key}: {value}")
    
    try:
        # Запуск обучения
        logger.info("\n🚀 Запуск обучения...")
        model, results = train_yolov13_dynamic_uav(**training_params)
        
        if results.get('success', False):
            logger.info("\n🎉 Обучение завершено успешно!")
            logger.info(f"📁 Модель сохранена: {results['model_path']}")
            
            # Отображение метрик
            best_metrics = results.get('best_metrics', {})
            if best_metrics:
                logger.info(f"📊 Лучшие метрики обучения:")
                for metric, value in best_metrics.items():
                    if isinstance(value, (int, float)):
                        logger.info(f"   {metric}: {value:.4f}")
                    else:
                        logger.info(f"   {metric}: {value}")
            
            # Тестирование модели
            try:
                logger.info("\n🧪 Тестирование обученной модели...")
                test_model = YOLO(results['model_path'])
                logger.info("✅ Модель успешно загружена")
                logger.info("🎯 Готова для обнаружения людей с UAV")
                
                # Валидация на приватном датасете (только в полном режиме)
                if not IS_DEBUG and os.path.exists(VAL_DATASET_PRIVATE):
                    logger.info("\n🔒 Финальная валидация на приватном датасете...")
                    final_private_results = validate_on_private_dataset(results['model_path'])
                    if 'error' not in final_private_results:
                        logger.info("📊 Результаты валидации на приватном датасете:")
                        for metric, value in final_private_results.items():
                            if isinstance(value, (int, float)):
                                logger.info(f"   {metric}: {value:.4f}")
                    else:
                        logger.warning(f"⚠️ Ошибка приватной валидации: {final_private_results['error']}")
                elif IS_DEBUG:
                    logger.info("🐛 Приватная валидация пропущена в режиме отладки")
                
            except Exception as e:
                logger.error(f"❌ Ошибка тестирования модели: {e}")
            
            # Рекомендации по использованию
            logger.info("\n📋 Рекомендации по использованию модели:")
            logger.info("   • Оптимальная высота съемки: 50-150м")
            logger.info("   • Рекомендуемое разрешение: 1920x1080 или выше")
            logger.info("   • Скорость полета: не более 15 м/с")
            logger.info("   • Угол камеры: 45-90° вниз")
            logger.info("   • Лучшие условия: дневное освещение, ясная погода")
            logger.info("   • Использование: model.predict('path_to_uav_image.jpg')")
            
            if IS_DEBUG:
                logger.info("\n🐛 Режим отладки завершен.")
                logger.info("   Для полного обучения установите IS_DEBUG = False")
                logger.info("   и перезапустите скрипт.")
            
            return model, results
            
        else:
            logger.error(f"\n❌ Обучение не завершено: {results.get('error', 'Неизвестная ошибка')}")
            return None, results
            
    except Exception as e:
        logger.error(f"\n💥 Критическая ошибка: {e}")
        import traceback
        traceback.print_exc()
        return None, {'error': str(e)}
        
    finally:
        
        
        logger.info("\n" + "=" * 70)
        logger.info("🏁 Завершение работы программы")

if __name__ == "__main__":
    # Запуск основной функции обучения YOLOv13 с Kaggle датасетами
    model, results = main()

2025-08-01 13:58:16,546 - MAIN - INFO - <cell line: 0>:97 - Logging setup completed
2025-08-01 13:58:16,558 - MAIN - INFO - main:1725 - 🚁 YOLOv13 Dynamic UAV - Система обнаружения людей для БПЛА
2025-08-01 13:58:16,559 - MAIN - INFO - main:1726 - ======================================================================
2025-08-01 13:58:16,561 - MAIN - WARNING - cleanup_debug_files:1002 - ⚠️ Ошибка очистки debug файлов: [Errno 16] Device or resource busy: '/kaggle/working'
2025-08-01 13:58:16,561 - MAIN - INFO - main:1733 - 🐛 РЕЖИМ ОТЛАДКИ АКТИВЕН
2025-08-01 13:58:16,562 - MAIN - INFO - main:1734 -    • Обучение на 800 изображениях
2025-08-01 13:58:16,563 - MAIN - INFO - main:1735 -    • 1 эпоха обучения
2025-08-01 13:58:16,564 - MAIN - INFO - main:1736 -    • Валидация на всех данных
2025-08-01 13:58:16,565 - MAIN - INFO - main:1750 - 
📊 Анализ доступных Kaggle датасетов...
2025-08-01 13:58:16,566 - MAIN - INFO - analyze_all_datasets:1015 - 📊 Анализ всех датасетов...
2025-08-01 13:58:16,5

[DEBUG] Logging setup completed successfully.


2025-08-01 13:58:34,553 - MAIN - INFO - collect_hierarchical_images:253 - ✅ Собрано 8484 пар изображение-метка из /kaggle/input/01trains1datasethumanrescu1
